In [2]:
# %pip install openai
# %pip install langchain
# %pip install langchain_community
# %pip install langchain_openai
# %pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Quiz App

- Given a page of a pdf
- Given a concept
- Simple interface
- Quickly generate flexible quizzes
  - Generate any type of questions (True or False, text based questions, multiple choice)
- Feedback on the answers

In [4]:
from openai import OpenAI

def get_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-4o", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a helpful assistant."},
                                 {"role": "user", "content": prompt}   
                             ],
                             temperature=0.0,
                             n = 1
                             )
    return response.choices[0].message.content


prompt = "Create a quiz about the basics of Python programming."
quiz_python = get_response(prompt)
quiz_python

'Sure! Here\'s a quiz designed to test basic knowledge of Python programming. Each question is followed by multiple-choice answers.\n\n### Python Basics Quiz\n\n#### Question 1:\nWhat is the correct file extension for Python files?\n\nA) .pyth  \nB) .pt  \nC) .py  \nD) .python\n\n#### Question 2:\nHow do you output "Hello, World!" in Python?\n\nA) echo "Hello, World!"  \nB) printf("Hello, World!")  \nC) print("Hello, World!")  \nD) cout << "Hello, World!";\n\n#### Question 3:\nWhich of the following is a correct variable assignment in Python?\n\nA) x := 10  \nB) int x = 10  \nC) x = 10  \nD) var x = 10\n\n#### Question 4:\nWhat is the correct way to create a list in Python?\n\nA) list = (1, 2, 3)  \nB) list = [1, 2, 3]  \nC) list = {1, 2, 3}  \nD) list = <1, 2, 3>\n\n#### Question 5:\nHow do you start a comment in Python?\n\nA) // This is a comment  \nB) <!-- This is a comment -->  \nC) # This is a comment  \nD) /* This is a comment */\n\n#### Question 6:\nWhich of the following is a v

In [5]:
from IPython.display import Markdown, display

Markdown(quiz_python)

Sure! Here's a quiz designed to test basic knowledge of Python programming. Each question is followed by multiple-choice answers.

### Python Basics Quiz

#### Question 1:
What is the correct file extension for Python files?

A) .pyth  
B) .pt  
C) .py  
D) .python

#### Question 2:
How do you output "Hello, World!" in Python?

A) echo "Hello, World!"  
B) printf("Hello, World!")  
C) print("Hello, World!")  
D) cout << "Hello, World!";

#### Question 3:
Which of the following is a correct variable assignment in Python?

A) x := 10  
B) int x = 10  
C) x = 10  
D) var x = 10

#### Question 4:
What is the correct way to create a list in Python?

A) list = (1, 2, 3)  
B) list = [1, 2, 3]  
C) list = {1, 2, 3}  
D) list = <1, 2, 3>

#### Question 5:
How do you start a comment in Python?

A) // This is a comment  
B) <!-- This is a comment -->  
C) # This is a comment  
D) /* This is a comment */

#### Question 6:
Which of the following is a valid function definition in Python?

A) function myFunction():  
B) def myFunction():  
C) define myFunction():  
D) func myFunction():

#### Question 7:
What is the output of the following code?

```python
x = 5
y = 10
print(x + y)
```

A) 15  
B) 510  
C) 5 + 10  
D) Error

#### Question 8:
Which of the following is used to handle exceptions in Python?

A) try/except  
B) catch/try  
C) handle/error  
D) error/catch

#### Question 9:
What is the output of the following code?

```python
for i in range(3):
    print(i)
```

A) 0 1 2  
B) 1 2 3  
C) 0 1 2 3  
D) 1 2

#### Question 10:
How do you create a dictionary in Python?

A) dict = [key1: value1, key2: value2]  
B) dict = {key1: value1, key2: value2}  
C) dict = (key1: value1, key2: value2)  
D) dict = <key1: value1, key2: value2>

---

### Answer Key:

1. C) .py
2. C) print("Hello, World!")
3. C) x = 10
4. B) list = [1, 2, 3]
5. C) # This is a comment
6. B) def myFunction():
7. A) 15
8. A) try/except
9. A) 0 1 2
10. B) dict = {key1: value1, key2: value2}

Feel free to use this quiz to test your knowledge or to help others learn the basics of Python programming!

Requirements for the app

1. We have to be specific
   1. We want to specify the number of questions we want
2. We have to be able to feed in contextual information
3. We want to be able to specify the types of questions we want created (T/F, multiple choice, text...)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [7]:
template = """
You are an expert quiz maker for technical fields. Let's think step by step and
create a quiz with {num_questions} {quiz_type} questions about the following concept/content: {quiz_context}.

The format of the quiz could be one of the following:
- Multiple-choice: 
- Questions:
    <Question1>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>
    <Question2>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>
    ....
- Answers:
    <Answer1>: <a|b|c|d>
    <Answer2>: <a|b|c|d>
    ....
    Example:
    - Questions:
    - 1. What is the time complexity of a binary search tree?
        a. O(n)
        b. O(log n)
        c. O(n^2)
        d. O(1)
    - Answers: 
        1. b
- True-false:
    - Questions:
        <Question1>: <True|False>
        <Question2>: <True|False>
        .....
    - Answers:
        <Answer1>: <True|False>
        <Answer2>: <True|False>
        .....
    Example:
    - Questions:
        - 1. What is a binary search tree?
        - 2. How are binary search trees implemented?
    - Answers:
        - 1. True
        - 2. False
- Open-ended:
- Questions:
    <Question1>: 
    <Question2>:
- Answers:    
    <Answer1>:
    <Answer2>:
Example:
    Questions:
    - 1. What is a binary search tree?
    - 2. How are binary search trees implemented?
    
    - Answers: 
        1. A binary search tree is a data structure that is used to store data in a sorted manner.
        2. Binary search trees are implemented using linked lists.
"""

In [8]:
prompt = ChatPromptTemplate.from_template(template)

In [9]:
prompt.format(num_questions=3, quiz_type="multiple-choice", quiz_context="Data Structures in Python Programming")

"Human: \nYou are an expert quiz maker for technical fields. Let's think step by step and\ncreate a quiz with 3 multiple-choice questions about the following concept/content: Data Structures in Python Programming.\n\nThe format of the quiz could be one of the following:\n- Multiple-choice: \n- Questions:\n    <Question1>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>\n    <Question2>: <a. Answer 1>, <b. Answer 2>, <c. Answer 3>, <d. Answer 4>\n    ....\n- Answers:\n    <Answer1>: <a|b|c|d>\n    <Answer2>: <a|b|c|d>\n    ....\n    Example:\n    - Questions:\n    - 1. What is the time complexity of a binary search tree?\n        a. O(n)\n        b. O(log n)\n        c. O(n^2)\n        d. O(1)\n    - Answers: \n        1. b\n- True-false:\n    - Questions:\n        <Question1>: <True|False>\n        <Question2>: <True|False>\n        .....\n    - Answers:\n        <Answer1>: <True|False>\n        <Answer2>: <True|False>\n        .....\n    Example:\n    - Questions:\n        

In [10]:
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
import re

In [11]:
chain = prompt | ChatOpenAI(temperature=0.0, model="gpt-4o") | StrOutputParser()

In [12]:
quiz_response = chain.invoke({"num_questions":3, "quiz_type":"multiple-choice", "quiz_context":"Data Structures in Python Programming",})

In [13]:
Markdown(quiz_response)

Let's create a multiple-choice quiz about Data Structures in Python Programming.

### Questions:

1. Which of the following data structures in Python is mutable?
    a. Tuple  
    b. List  
    c. String  
    d. Frozenset  

2. What is the primary characteristic of a Python dictionary?
    a. It is an ordered collection of elements.  
    b. It allows duplicate keys.  
    c. It is a collection of key-value pairs.  
    d. It is immutable.  

3. Which of the following operations is the most efficient for a Python set?
    a. Accessing an element by index  
    b. Checking for membership  
    c. Appending an element  
    d. Sorting the elements  

### Answers:

1. b  
2. c  
3. b  

In [14]:
quiz_response = chain.invoke({"num_questions": 4, "quiz_type": "True-false", "quiz_context": "Python programming"})

In [15]:
Markdown(quiz_response)

Let's create a True-false quiz about Python programming. We'll think through each question to ensure it covers a fundamental aspect of Python.

### Questions:
1. Python is a statically typed programming language: True|False
2. In Python, indentation is used to define the scope of loops, functions, and classes: True|False
3. Python supports multiple inheritance: True|False
4. The `print` function in Python automatically adds a newline character at the end of its output: True|False

### Answers:
1. False
2. True
3. True
4. True

### Explanation:
1. **False**: Python is a dynamically typed language, meaning that variable types are determined at runtime.
2. **True**: Python uses indentation to define the scope of code blocks, such as loops, functions, and classes, rather than braces or keywords.
3. **True**: Python supports multiple inheritance, allowing a class to inherit from more than one parent class.
4. **True**: By default, the `print` function in Python adds a newline character at the end of its output, which can be changed by setting the `end` parameter.

In [16]:
quiz_response = chain.invoke({"num_questions": 2, "quiz_type": "Open-ended", "quiz_context": "Python programming"})

In [17]:
Markdown(quiz_response)

Sure, let's create a quiz with two open-ended questions about Python programming.

### Questions:
1. What are Python decorators, and how are they used in a Python program?
2. Explain the concept of list comprehensions in Python and provide an example of how they can be used to create a new list.

### Answers:
1. Python decorators are a powerful tool that allows you to modify the behavior of a function or class. They are functions that take another function as an argument and extend or alter its behavior. Decorators are often used for logging, access control, instrumentation, and caching. They are applied using the `@decorator_name` syntax above the function definition.

2. List comprehensions in Python provide a concise way to create lists. They consist of an expression followed by a `for` clause, and can include additional `for` or `if` clauses. The result is a new list resulting from evaluating the expression in the context of the `for` and `if` clauses. For example, `[x**2 for x in range(10) if x % 2 == 0]` creates a list of squares of even numbers from 0 to 9, resulting in `[0, 4, 16, 36, 64]`.